# 배당 X1.5

In [ ]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt

initial_money = 100000
money = initial_money
profits = [money]

min_train_size = 100
bet_threshold = 0.7
bet_amount = 1000

# 저장할 기록 리스트
records = []

for i in range(min_train_size, len(df) - 1):
    X_train = X.iloc[:i]
    y_train = y.iloc[:i]
    X_test = X.iloc[i:i+1]
    y_test = y.iloc[i:i+1]
    odds = df.iloc[i]['배당률']
    game_date = df.iloc[i].get('날짜', i)

    # 전처리
    X_train_processed = preprocessing.fit_transform(X_train)
    X_test_processed = preprocessing.transform(X_test)

    # 모델 생성 및 학습
    base_models = create_base_models()
    stacking_model = create_stacking_classifier(base_models)
    stacking_model.fit(X_train_processed, y_train)

    # 예측
    prob = stacking_model.predict_proba(X_test_processed)[0, 1]
    true = y_test.values[0]

    # 기록 저장 (베팅 여부, 수익은 제외)
    records.append({
        "날짜": game_date,
        "예측확률": prob,
        "실제결과": true,
        "배당률": odds
    })

    # 베팅 로직 (이건 실행 중에만 사용됨)
    if prob >= bet_threshold:
        if true == 1:
            money += odds * bet_amount - bet_amount
        else:
            money -= bet_amount
    profits.append(money)

    # 출력
    if (i - min_train_size) % 10 == 0 or prob >= 0.9:
        print(f"[경기 {i}] 날짜: {game_date}, 예측확률: {prob:.3f}, 실제결과: {true}, 배당률: {odds}, 누적자본: {money:,.0f}원")

# 결과 저장 (Pickle)
result_df = pd.DataFrame(records)
result_df.to_pickle("model_prediction_results.pkl")

# 그래프
plt.figure(figsize=(10, 5))
plt.plot(profits, label="누적 수익")
plt.axhline(initial_money, color='red', linestyle='--', label="초기 자본")
plt.xlabel("경기 수")
plt.ylabel("자본")
plt.title("경기 단위 베팅 전략 수익률 시뮬레이션")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

print(f"\n최종 수익률: {(money - initial_money) / initial_money * 100:.2f}%")
print(f"최종 자본: {money:,.0f}원")
print("📁 예측 결과가 'model_prediction_results.pkl'로 저장되었습니다.")
